# Introduction
* Penjelasan tentang proyek AI Chatbot untuk regulasi Dana BOS.
* Tujuan: Membantu pengguna mencari informasi tentang Dana BOS berdasarkan regulasi yang tersedia.
* Metode: Menggunakan NLP untuk ekstraksi teks dan pencarian semantik dalam dokumen.

# Import Library

In [1]:
# Install library yang dibutuhkan
!pip install pymupdf nltk sentence-transformers

# Import library
import os
import re
import json
import fitz  # PyMuPDF untuk ekstraksi teks dari PDF
import torch
import nltk
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
from google.colab import drive

# Download NLTK tokenizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Gathering

In [2]:
# Data Gathering
# ===============================
# 1. MOUNT GOOGLE DRIVE & CEK FILES
# ===============================

# Mount Google Drive
drive.mount('/content/drive')

# Path ke direktori penyimpanan file PDF
pdf_dir = "/content/drive/My Drive/Colab Notebooks/AI Chatbot Berbasis Regulasi"

# Cek apakah direktori ada
if not os.path.exists(pdf_dir):
    raise FileNotFoundError(f"Direktori {pdf_dir} tidak ditemukan! Periksa kembali path-nya.")
else:
    print(f"Direktori ditemukan! Daftar file PDF: {os.listdir(pdf_dir)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Direktori ditemukan! Daftar file PDF: ['PERMENDIKBUD 6 TAHUN 2021 - BOS REGULER.pdf', 'Permendikbudriset No. 63 Tahun 2023.pdf', 'PERMENDIKBUDRISTEK NOMOR 63 TAHUN 2022.pdf', 'Kepmen nomor 211 P 2024 Penerima BOP PAUD BOS kinerja BOP Kesetaraan kinerja 2024.pdf', 'extracted_text.json']


In [3]:
# ===============================
# 2. EKSTRAKSI TEKS DARI FILE PDF
# ===============================

def extract_text_from_pdf(pdf_path):
    """Mengekstrak teks dari file PDF."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

# Ambil semua file PDF dari folder
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]

# Dictionary untuk menyimpan hasil ekstraksi teks
pdf_texts = {}

# Loop ekstraksi teks untuk setiap file PDF
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    pdf_texts[pdf_file] = text
    print(f"Teks berhasil diekstrak dari: {pdf_file}")

Teks berhasil diekstrak dari: PERMENDIKBUD 6 TAHUN 2021 - BOS REGULER.pdf
Teks berhasil diekstrak dari: Permendikbudriset No. 63 Tahun 2023.pdf
Teks berhasil diekstrak dari: PERMENDIKBUDRISTEK NOMOR 63 TAHUN 2022.pdf
Teks berhasil diekstrak dari: Kepmen nomor 211 P 2024 Penerima BOP PAUD BOS kinerja BOP Kesetaraan kinerja 2024.pdf


In [4]:
# ===============================
# 3. SIMPAN TEKS DALAM FILE JSON
# ===============================

json_path = os.path.join(pdf_dir, "extracted_text.json")
with open(json_path, "w", encoding="utf-8") as json_file:
    json.dump(pdf_texts, json_file, ensure_ascii=False, indent=4)

print(f"Hasil ekstraksi teks disimpan dalam: {json_path}")

with open(json_path, "r", encoding="utf-8") as json_file:
    extracted_data = json.load(json_file)

# Menampilkan sebagian isi JSON
for pdf, text in extracted_data.items():
    print(f"\n=== {pdf} ===\n")
    print(text[:1000])  # Menampilkan 1000 karakter pertama

Hasil ekstraksi teks disimpan dalam: /content/drive/My Drive/Colab Notebooks/AI Chatbot Berbasis Regulasi/extracted_text.json

=== PERMENDIKBUD 6 TAHUN 2021 - BOS REGULER.pdf ===

https://jdih.kemdikbud.go.id 
SALINAN 
MENTERI PENDIDIKAN DAN KEBUDAYAAN 
REPUBLIK  
 
 
 
RANCNGAN 
PERATURAN MENTERI PENDIDIKAN DAN KEBUDAYAAN 
REPUBLIK INDONESIA 
NOMOR   6  TAHUN 2021 
TENTANG 
PETUNJUK TEKNIS PENGELOLAAN  
DANA BANTUAN OPERASIONAL SEKOLAH REGULER  
 
DENGAN RAHMAT TUHAN YANG MAHA ESA 
 
MENTERI PENDIDIKAN DAN KEBUDAYAAN REPUBLIK INDONESIA, 
 
 
Menimbang 
:   a. 
bahwa untuk meningkatkan mutu pembelajaran dan 
pemerataan 
akses 
layanan 
Pendidikan, 
perlu 
mengalokasikan 
dan 
menyalurkan 
dana 
bantuan 
operasional sekolah reguler; 
b. 
bahwa untuk mendukung pengelolaan dana bantuan 
operasional sekolah reguler secara akuntabel dan tepat 
sasaran, perlu menyusun petunjuk teknis pengelolaan 
dana bantuan operasional sekolah reguler; 
c. 
bahwa Peraturan Menteri Pendidikan dan Kebudayaan

# Preprocessing Data

In [5]:
# ===============================
# 4. PREPROCESSING TEKS
# ===============================

def clean_text(text):
    """Membersihkan teks dari karakter khusus dan spasi berlebih."""
    text = re.sub(r'\n+', ' ', text)  # Hapus newline berlebih
    text = re.sub(r'\s+', ' ', text).strip()  # Hapus spasi berlebih
    return text

# Terapkan pembersihan ke semua teks yang telah diekstraksi
cleaned_texts = {pdf: clean_text(text) for pdf, text in pdf_texts.items()}

# Training Model

In [6]:
# ===============================
# 5. TOKENISASI & EMBEDDING TEKS
# ===============================

# Gabungkan semua teks untuk pembelajaran model
all_text = " ".join(cleaned_texts.values())

# Tokenisasi kalimat
sentences = sent_tokenize(all_text)

# Load model Sentence Transformer untuk embedding teks
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Konversi kalimat menjadi vektor menggunakan model
sentence_embeddings = model.encode(sentences, convert_to_tensor=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Testing

In [7]:
# ===============================
# 6. TESTING CHATBOT
# ===============================

def search_query(query, sentences, model, sentence_embeddings):
    """Mencari jawaban terbaik berdasarkan query menggunakan model embedding."""
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, sentence_embeddings)
    best_match_idx = torch.argmax(similarities).item()
    return sentences[best_match_idx]

# Contoh pertanyaan pengguna
question = "Apa saja syarat penerima dana BOS?"
answer = search_query(question, sentences, model, sentence_embeddings)

print("\nPertanyaan:", question)
print("Jawaban:", answer)


Pertanyaan: Apa saja syarat penerima dana BOS?
Jawaban: Bagian Keempat Penetapan Penerima Dana BOSP Pasal 16 Penerima Dana BOSP yang memenuhi persyaratan sebagaimana dimaksud dalam Pasal 5, Pasal 6, Pasal 8, Pasal 10, Pasal 11 ayat (1), Pasal 12 ayat (1), Pasal 14, Pasal 15 ayat (1) ditetapkan dengan Keputusan Menteri untuk setiap tahun anggaran.
